In [1]:
# -*- coding: utf-8 -*-
from Code import recommendation
from imp import reload
# reload(Code.recommendation)

/home/zijun.wu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
reload(recommendation)

In [1]:
import numpy as np
import pandas as pd
import pickle
import sys
import inspect
import multiprocessing as mp
import pymysql
import configparser
from scipy.sparse import coo_matrix
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, jaccard
from sklearn.metrics import pairwise_distances

In [ ]:
1

In [ ]:
config = configparser.ConfigParser()
config.read("../Data/Input/database_config/database.conf")
host = config['ASSESSMENT']['host']
user = config['ASSESSMENT']['user']
password = config['ASSESSMENT']['password']
database = config['ASSESSMENT']['database']
port = config['ASSESSMENT']['port']
charset = config['ASSESSMENT']['charset']
db = pymysql.connect(host=host, user=user, password=password, db=database, port=int(port), charset=charset)

# 尝试测试

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
def simple_minmax(column_target, min_v=0.001, max_v=1):
    target = column_target.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(min_v, max_v))
    scaler.fit(target)
    return scaler.transform(target)

In [ ]:
tag_tag = pd.read_csv("../Data/Output/Tag_graph/temp_result/tag_tag_result_all", header=None)

In [ ]:
tag_dict = pickle.load(open("../Data/Output/Tag_graph/v1/tag_dict.pkl", "rb"))

In [ ]:
tag_tag.columns = ["tag1", "tag2", "link_type", "link_value"]
tag_tag.link_value = tag_tag.link_value.apply(lambda x: np.log2(min(0.000001 + x, 1)))
tag_tag.link_value = simple_minmax(tag_tag.link_value)
tag_tag = tag_tag[["tag1", "tag2", "link_type", "link_value"]].drop_duplicates()
tag_tag["tag_link"] = tag_tag.tag1 + "-" + tag_tag.tag2

In [ ]:
tag_tag_weights=(0.5, 0.3, 0.2)

In [ ]:
tag_index = dict(zip(tag_dict.values(), range(0, len(tag_dict))))

In [ ]:
length = len(tag_index)

In [ ]:
length

In [ ]:
row = np.array(tag_tag.tag1.apply(lambda x: tag_index.get(x)))

In [ ]:
col = np.array(tag_tag.tag2.apply(lambda x: tag_index.get(x)))

In [ ]:
data = np.array(tag_tag.link_value * tag_tag.link_type.apply(lambda x: tag_tag_weights[x]))

In [ ]:
pickle.dump(tag_tag_sparse, open("../Data/Output/Tag_graph/tag_tag_sparse.pkl", "wb"))

In [ ]:
tag_tag_sparse = pickle.load(open("../Data/Output/Tag_graph/tag_tag_sparse.pkl", "rb"))

In [ ]:
comp_tags_all = pickle.load(open("../Data/Output/Tag_graph/comp_tags_all.pkl", "rb"))

In [ ]:
comp_tags_all.loc[comp_tags_all.comp_id == '10000007422129887091'].index.tolist()[0]

## 生成公司-标签向量全量稀疏矩阵

In [ ]:
comp_tags_all["row_num"] = comp_tags_all.index

In [ ]:
comp_tags_all["coo"]= comp_tags_all[["comp_tag_list", "row_num"]].apply(lambda x: [(x[1], tag_index.get(w)) for w in x[0]], axis=1)

In [ ]:
coord_list = [x for y in comp_tags_all.coo for x in y]

In [ ]:
test = zip(*coord_list)

In [ ]:
coords = tuple(map(list, zip(*coord_list)))

In [ ]:
comp_tags_all_sparse = coo_matrix((np.ones(len(coord_list), dtype=int), (coords[0], coords[1])))

In [ ]:
pickle.dump(comp_tags_all_sparse, open("../Data/Output/Tag_graph/comp_tags_all_sparse", "wb"))

In [ ]:
comp_tags_all_sparse, tag_tag_sparse

## 计算测试

In [ ]:
first = tag_tag_sparse.dot(test.T)

In [ ]:
%time x = comp_tags_all_sparse.dot(tag_tag_sparse.dot(test.T))

In [ ]:
y = x.toarray()

In [ ]:
y

In [ ]:
t = 1 - pairwise_distances(comp_tags_all_sparse.tocsr(), test.tocsr(), metric='cosine')

In [ ]:
merged = np.concatenate((x.toarray(), t), axis=1)

In [ ]:
comp_row_num = dict(zip(comp_tags_all.comp_id, comp_tags_all.row_num))

In [ ]:
tag_list = list(tag_index.keys())

## 计算函数

In [ ]:
tag_dict_reverse = recommendation.tag_dict_reverse

In [ ]:
comp_name = "郑州宇通客车股份有限公司"

In [ ]:
%time test = recommendation.sparse_cal(comp_name, response_num=200)

In [ ]:
test[2].to_excel("宇通客车.xlsx")

## 平安20家公司

In [ ]:
has_invest = samples[samples.最近融资时间 != 0].sample(10)
no_invest = samples[samples.最近融资时间 == 0].sample(10)

In [ ]:
comp_infos, tag_tag, comp_id_name, tag_dict, comp_tags_all_sparse, tag_tag_sparse = recommendation.comp_infos, recommendation.tag_tag, recommendation.comp_id_name, recommendation.tag_dict, recommendation.comp_tags_all_sparse, recommendation.tag_tag_sparse

In [ ]:
comp_list = ["深圳市移联网络科技有限公司", "康泽药业股份有限公司", "深圳市泛谷药业股份有限公司", 
              "广东瑞兴医药股份有限公司", "深圳市微购科技有限公司", "快尚时装（广州）有限公司", 
              "广州食得鲜网络科技有限公司", "百草堂医药股份有限公司", "广州启生信息技术有限公司", 
              "深圳市百分之百数码科技有限公司", "深圳市金檀科技有限公司", "广州诚泽信息科技有限公司", "深圳市世纪润和珠宝有限公司", 
             "深圳市佰供信息技术有限公司", "中山市放心大药房有限公司", "佛山市居道家具有限公司", 
             "中山市辰星医药有限公司", "广州美妆广告有限公司", "东莞市万顺建材有限公司", 
             "佛山市盈博悦贸易有限公司"]

In [ ]:
def get_competitors(comp_list):
    target_comp_tags = []
    no_records = []
    for comp_name in comp_list:
        print("开始计算 %s" % comp_name)
        result = recommendation.sparse_cal(comp_name, response_num=50)
        if result != -1:
            result_list = tuple(result[2].comp_id.tolist())
            # print(result_list)
            sql_intro = "select distinct comp_id, comp_full_name, intro  \
                from company_intro_info_latest0703 where comp_id in %s" % str(result_list)
            intro = pd.read_sql(sql_intro, con=db)
            # print(intro.head(3))
            result[2].merge(intro, how="left", on="comp_id").to_excel(comp_name + ".xlsx")
            target_comp_tags.append((result[0], comp_name, result[1]))
            print ("%s 计算完成" % comp_name)
        else:
            no_records.append(comp_name)
    targets_tags = pd.DataFrame(target_comp_tags, columns=["comp_id", "comp_name", "tags"])
    targets_info_sql = "select distinct comp_id, comp_full_name, intro  \
            from company_intro_info_latest0703 where comp_id in %s" % str(tuple(targets_tags.comp_id.tolist()))
    targets_intros = pd.read_sql(targets_info_sql, con=db)
    targets_tags.merge(targets_intros, how="left", on="comp_id").to_excel("target_comp_tags.xlsx")
    if len(no_records):
        print("以下公司未在库中匹配到：%s" % ",".join(no_records))
    return 0

## 首部1500家抽取

In [ ]:
top_comps = pd.read_excel("潜力企业数据.xlsx")

In [ ]:
length = len(top_comps)
layer_deep = length // 3
rand_list = np.random.choice(layer_deep, size=10, replace=False)

In [ ]:
sample_index = np.concatenate((rand_list, rand_list + layer_deep, rand_list + 2*layer_deep))

In [ ]:
rand_list

In [ ]:
comp_name_list = top_comps.loc[sample_index]["公司全称"].tolist()

In [ ]:
comp_name_list

In [ ]:
Counter(comp_name_list)

In [ ]:
%time get_competitors(comp_name_list)

In [ ]:
len(set(comp_name_list))

## 发现产标签公司的影响实在太大，尝试不对tag-tag关联值进行log处理

In [ ]:
tag_tag_weights=(0.5, 0.3, 0.2)

In [ ]:
def simple_minmax(column_target, min_v=0.001, max_v=1):
    target = column_target.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(min_v, max_v))
    scaler.fit(target)
    return scaler.transform(target)

In [ ]:
tag_tag = pd.read_csv("../Data/Output/Tag_graph/temp_result/tag_tag_result_all", header=None)
tag_dict = tag_dict = pickle.load(open("../Data/Output/Tag_graph/tag_dict.pkl", "rb"))
tag_tag.columns = ["tag1", "tag2", "link_type", "link_value"]
# tag_tag.link_value = tag_tag.link_value.apply(lambda x: np.log2(min(0.000001 + x, 1)))
tag_tag.link_value = simple_minmax(tag_tag.link_value)
tag_tag = tag_tag[["tag1", "tag2", "link_type", "link_value"]].drop_duplicates()
tag_index = dict(zip(tag_dict.values(), range(0, len(tag_dict))))
length = len(tag_index)
row = np.array(tag_tag.tag1.apply(lambda x: tag_index.get(x)))
col = np.array(tag_tag.tag2.apply(lambda x: tag_index.get(x)))
data = np.array(tag_tag.link_value * tag_tag.link_type.apply(lambda x: tag_tag_weights[x]))
tag_tag_sparse = coo_matrix((data, (row, col)), shape=(length, length))
pickle.dump(tag_tag_sparse, open("../Data/Output/Tag_graph/tag_tag_no_log_sparse.pkl", "wb"))

In [ ]:
data.max()

## 接口测试

In [ ]:
import requests
import json
import pandas as pd
import pickle

In [8]:
comp_name = "湖北优耐德节能环保科技有限公司"
comp_info = "传统行业,制造业,材料类,建筑业,装配式建筑#1,2,3,4,4"

In [3]:
comp_full_name = "北京必果科技有限公司"
comp_info2 = "英语口语,听说读写,新零售,语言学习,教育,移动互联网教育,生活场景,少儿外语,互动设计,英语学习,学习app,便利店,智能卡,儿童英语,学英语,儿童英语学习,卡通#4,3,1,3,3,4,1,3,1,3,2,1,1,4,3,4,1"
top_n_comp = 100

In [3]:
metric = "cosine"

In [9]:
test = recommendation.sparse_cal(comp_name=comp_name, metric=metric, comp_info=comp_info, response_num=100)

In [10]:
test

-1

In [5]:
test2 = recommendation.sparse_cal(comp_name=comp_full_name, metric=metric, comp_info=comp_info2, response_num=100)

In [ ]:
test2[2]

In [ ]:
%time r = requests.post("http://172.30.212.219:8999/recommendation", data={"comp_name": comp_name, "metric": "euclidean", "comp_info": "", "response_num": 20})

In [ ]:
%time r2 = requests.post("http://172.30.212.219:8999/recommendation", data={"comp_name": comp_full_name, "comp_info": comp_info2, "response_num": top_n_comp})

In [ ]:
result_df = pd.DataFrame(r.json().get("data"))

In [ ]:
result_df

In [ ]:
comp_id_list = list(set(pd.read_excel("companies.xlsx")["公司id"]))

In [ ]:
comp_name_list = list(set(pd.read_excel("companies.xlsx")["公司名"]))

In [ ]:
def get_competitors_from_api(comp_name, comp_info=""):
    result = requests.post("http://172.29.237.212:8999/recommendation", data={"comp_name": comp_name, "comp_info": comp_info, "response_num": 50})
    result_df = pd.DataFrame(result.json().get("data"))
    result_df.to_excel("310公司/%s.xlsx" % comp_name)

In [ ]:
for i in range(len(comp_name_list)):
    print("#### %d start %s #####" % (i, comp_name_list[i]))
    get_competitors_from_api(comp_name_list[i])
    print("#### save %s ####" % comp_name_list[i])

## 指定目标和匹配池进行相似度计算

In [1]:
from Code.recommendation import *

/home/zijun.wu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [34]:
inputs = pd.read_excel("相似度计算 update.xlsx")

In [35]:
target_comps = inputs.columns.tolist()
search_pool = inputs.index.tolist()

In [36]:
def matrix_similarities(target_comps, search_pool):
    target_ids = comp_id_name[comp_id_name.name.isin(target_comps)].point_id.tolist()
    search_pool_ids = comp_id_name[comp_id_name.name.isin(search_pool)].point_id.tolist()
    
    target_indexes = comp_infos[comp_infos.comp_id.isin(target_ids)].index.tolist()
    search_pool_indexes = comp_infos[comp_infos.comp_id.isin(search_pool_ids)].index.tolist()
    # print(len(target_ids), len(search_pool_ids), len(target_indexes), len(search_pool_indexes))
    
    column_ids = comp_infos.loc[target_indexes].comp_id.to_frame()
    index_ids = comp_infos.loc[search_pool_indexes].comp_id.to_frame()
    column_name = column_ids.merge(comp_id_name, left_on="comp_id", right_on="point_id").name.tolist()
    index_name = index_ids.merge(comp_id_name, left_on="comp_id", right_on="point_id").name.tolist()

    csr_matrix = comp_tags_all_sparse.tocsr()
    target_matrix = csr_matrix[target_indexes, :]
    search_pool_matrix = csr_matrix[search_pool_indexes]
    udf_sim = search_pool_matrix.dot(tag_tag_sparse.dot(target_matrix.T)).toarray()
    cosine_sim = 1 - pairwise_distances(search_pool_matrix, target_matrix, metric="cosine")

    sim_prod = cosine_sim * udf_sim
    result = pd.DataFrame(sim_prod, index=index_name, columns=column_name)
    return result

In [37]:
r = matrix_similarities(target_comps, search_pool)

In [38]:
r.to_excel("similarities_result_update.xlsx")

## 主营结果统计分析

In [ ]:
main_business = pd.read_sql("select comp_id, comp_full_name, info_extra_tag, name_tag from innotree_data_assessment.comp_main_business_0726", con=db)

In [ ]:
main_business.fillna("", inplace=True)

In [ ]:
main_business.head()

In [ ]:
main_business[["info_extra_tag", "name_tag"]] = main_business.apply({"info_extra_tag": lambda x: x.strip("\n"), "name_tag": lambda x: x.strip("\n")})

In [ ]:
main_business.head(3)

In [ ]:
main_business["extra_tag_count"] = main_business.info_extra_tag.apply(lambda x: 0 if x == "" else len(set(x.split("-"))))

In [ ]:
main_business["name_tag_count"] = main_business.name_tag.apply(lambda x: 0 if x == "" else len(set(x.split("-"))))

In [ ]:
main_business.describe()